# MLPMixer

MLP Mixer 是 2021 年由 Google Brain 团队发表的文章，文章主要是用一个全是 MLP 的 Block 结构来替换原来的 CNN 的结构或者 ViT 中的 Transformer 中的 SelfAttention 的结构。

论文的摘要：

卷积神经网络（CNN）是计算机视觉领域的首选模型。最近，基于注意力机制的网络（如Vision Transformer）也逐渐流行起来。在本文中，我们展示了虽然卷积和注意力机制都足以取得良好性能，但它们都不是必要的。我们提出了MLP-Mixer，这是一种完全基于多层感知机（MLP）的架构。MLP-Mixer包含两种类型的层：一种是将MLP独立应用于图像块（即“混合”每个位置的特征），另一种是将MLP跨图像块应用（即“混合”空间信息）。当在大规模数据集上进行训练，或使用现代正则化方案时，MLP-Mixer在图像分类基准测试中获得了具有竞争力的成绩，其预训练和推理成本与当前最先进的模型相当。我们希望这些结果能够激发对CNN和Transformer以外领域的进一步研究。

整个论文的核心实际就是下面这张图：

<img src="../images/mlp-mixer-arch.png" width=800px>

整个 MLP-Mixer 的网络中包括了一个基于 Patch 的 Linear embedding，一些 Mixer Layers 和一个分类头构成。其中的 Mixer Layer 的主要组成部分是 token-mixing MLP 以及 channle-mixing MLP。

Token Mixing 是执行在 patches 维度上的 MLP，可以看成是在每个通道维上将所有patch 位置进行融合。输出的 hidden state 中每个 patch 位置上的信息 融合自了输入的所有 patch 上的信息，这是区别于 Depth-wise Conv的地方。但 Token Mixing中不同的 channel 上用的 MLP 的权重是一样的，这也是和 Depth-wise Conv 不同的地方。

Channel-Mixing 可以看作是在执行通道维度上的信息融合。

In [5]:
from timm.layers import Mlp, DropPath, to_2tuple
from functools import partial
from torch import nn

In [6]:
class MixerBlock(nn.Module):
    """Residual Block w/ token mixing and channel MLPs
    Based on: 'MLP-Mixer: An all-MLP Architecture for Vision' - https://arxiv.org/abs/2105.01601
    """

    def __init__(
        self,
        dim,
        seq_len,
        mlp_ratio=(0.5, 4.0),
        mlp_layer=Mlp,
        norm_layer=partial(nn.LayerNorm, eps=1e-6),
        act_layer=nn.GELU,
        drop=0.0,
        drop_path=0.0,
    ):
        super().__init__()
        tokens_dim, channels_dim = [int(x * dim) for x in to_2tuple(mlp_ratio)]
        self.norm1 = norm_layer(dim)
        self.mlp_tokens = mlp_layer(seq_len, tokens_dim, act_layer=act_layer, drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()
        self.norm2 = norm_layer(dim)
        self.mlp_channels = mlp_layer(dim, channels_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        # x.shape: [batch_size, patches, channels]

        # norm -> transpose -> mlp -> transpose 回来
        x = x + self.drop_path(
            self.mlp_tokens(self.norm1(x).transpose(1, 2)).transpose(1, 2)
        )
        x = x + self.drop_path(self.mlp_channels(self.norm2(x)))
        return x